# PyVG: Data Science to predict Video Games sales
>Equipe: Alexis Terrasse, Henri-François Mole, Hsan Drissi, Stephane Lelievre
>
>Promo: DS_Oct21
---
## Scraping Metacritic - Alexis
<img src='https://earlygame.com/uploads/images/_800x418_crop_center-center_82_none/metacritic.jpg?mtime=1591260456' width=500></img>

### Importation des bibliothèques

In [ ]:
from bs4 import BeautifulSoup # Importation de BeautifulSoup
import requests # Importation de import requests 
from time import sleep # Importation de sleep
import pandas as pd # Importation de pandas sous l'alias pd
from tqdm import tqdm # Importation de tqdm
import datetime # Importation de datetime
import os # Importation de os

### Définition de fonctions

In [ ]:
##################################################################
######### Fonction de récupération des données ddu jeu  ##########
##################################################################

def retrieve_data(sp): 
    
    # Fonction de récupération du Nom, de la plateforme, de l'éditeur, 
    # de l'année de sortie, du/des genres, du metascore et du userscore 
            
    span= sp.select("span[class=data]") # On récupère les balises span contenant class=data dans une liste
    
    g_name= sp.select('h1')[0].text.strip() # On récupère le titre du jeu 
    
    g_platform= sp.select('.platform')[0].text.strip() # On récupère la plateforme

    # On récupère l'éditeur 
    if sp.select('.publisher a') != [] :
        g_publisher= sp.select('.publisher a')[0].text.strip()
    else :
        g_publisher=None
      
    # On récupère l'année de sortie
    if sp.select('.release_data .data') != [] :
        g_year=sp.select('.release_data .data')[0].text.strip()
    else :
        g_year=None    
        
    # On récupère le développeur lorsqu'il est dispo
    if sp.select('.developer .label') != [] :
        g_developer=sp.select('.button')[0].text.strip()
    else :
        g_developer=None
        
    # Pour le genre comme il peut y en avoir plusieurs on les concatène après les avoir récupèré
    g_genre= ', '.join(sp.find_all('li',{'class' : 'summary_detail product_genre'})[0].text[9:].strip().split(',                                            '))
    
    
    # Pour le metascore on vérifie s'il la valeur existe si ce n'est pas le cas alors g_metas reçoit None
    g_metas= -1
    for tag in sp.select('span',{'class':'desc'}):
        if 'No score yet' in tag.text:
            g_metas= None
    if g_metas != None :
        g_metas= int(sp.select("a[class^=metascore_anchor]")[0].text) #Sinon on récupère le metascore
    
    # Pour le userscore on vérifie s'il reste à déterminer 'tbd'
    if sp.select("a[class^=metascore_anchor]")[1].text.strip() == 'tbd':
        g_users= None #Si oui alors on lui affecte Non
    else : #Sinon s'il n'y a pas de g_metas on affecte la valeur du premier élément de la liste sp.select("a[class^=metascore_anchor]")
        if g_metas == None: g_users= float(sp.select("a[class^=metascore_anchor]")[0].text)
        else : g_users= float(sp.select("a[class^=metascore_anchor]")[1].text)  #Sinon on lui affecte le second élémen
            
            
    if sp.select('.count a span') != [] :
        g_n_pro_crit= int(sp.select('.count a span')[0].text.strip())
    else :
        g_n_pro_crit= 0          
            
    if sp.select('.feature_userscore a') != [] :
        if len(sp.select('.feature_userscore a')) > 1:
            g_n_user_crit= int(sp.select('.feature_userscore a')[1].text.strip()[:-8])
        else : g_n_user_crit= 0
    else :
        g_n_user_crit= 0           
            
            
    #On retroune les différentes valeurs trouvées
    return(g_name,g_platform,g_publisher,g_year,g_developer,g_genre,g_metas,g_users,g_n_pro_crit,g_n_user_crit ) 


##################################################################
########## Fonction de récupération des critiques pro  ###########
##################################################################

def retrieve_pro_critics(url):     
    # Fonction de récupération des meta reviews (critiques pro)
    
    crit_url= url+'/critic-reviews' # on se place sur la page des critques pro
    headermap = {"User-Agent": "Mac Firefox"};
    crit_markup = requests.get(crit_url, headers=headermap, timeout=60).text # On requête le serveur
    crit_soup= BeautifulSoup(crit_markup,"lxml") #On parse le résultat avec BeautifulSoup
    
    #On détermine le nombre de pages pour les reviews pro
    if len(crit_soup.select("ul[class=pages]")) == 0 : 
        crit_pages = 0
    else : 
        crit_pages= int(crit_soup.select("ul[class=pages]")[0].select("a[class=page_num]")[-1].text)
    
    #On instancie une liste vide dans laquelle on stockera les infos de chaque reviews
    crit_review=[]   
    
    if crit_pages == 0 : # S'il n'y a qu'une page 
        
        # On parcours toutes les critiques de la page (trouvées dans les balises crit_soup.select("div[class^=review_content]")[:-3])
        for review in crit_soup.select("div[class^=review_content]")[:-3] :
            if review.find('div', class_='name') == None: break # Si on trouve rien on s'arrête
                
            #Récupération de la date de la review
            if review.select("div[class^=date]") != [] : #Si on trouve une balise avec la date 
                review_date= review.select("div[class^=date]")[0].text # On l'affecte à la variable review_date
            else : review_date= 'None' # Sinon on affecte None à review_date
                
            review_name= review.select("div[class^=source]")[0].text # Récupération du nom du critique
            review_score= review.select("div[class^=metascore_w]")[0].text # Récupération du score donné par le critique
            review_body= review.select("div[class^=review_body]")[0].text.strip() # Récupération du commentaire du critique
            crit_review.append([review_date,review_name,review_score,review_body]) # On ajoute les infos à la liste crit_review
            
    else:  
        # S'il y a plusieurs pages on répète l'opération ci-dessous mais pour chaque page
        for i in range(0,len(crit_pages)+1):
            
            crit_url= url+'/critic-reviews&page='+str(i)
            headermap = {"User-Agent": "Mac Firefox"};
            crit_markup = requests.get(crit_url, headers=headermap,timeout=60).text
            crit_soup= BeautifulSoup(markup,"lxml")
            
            for review in crit_soup.select("div[class^=review_content]")[:-3] :
                if review.select("div[class^=date]") != []:             
                    review_date= review.select("div[class^=date]")[0].text
                else : review_date= 'None'
                review_name= review.select("div[class^=source]")[0].text
                review_score= review.select("div[class^=metascore_w]")[0].text
                review_body= review.select("div[class^=review_body]")[0].text.strip()
                crit_review.append([review_date,review_name,review_score,review_body])
                
    return crit_review # On retourne la liste crit_review avec toutes les infos trouvées

#########################################################################################

##################################################################
########## Fonction de récupération des critiques pro  ###########
##################################################################

def critic_reviews(url):
    # Fonction de récupération des meta reviews (critiques pro)
     
    url= url+'/critic-reviews'  # on se place sur la page des critques pro
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent, timeout=60) # On requête le serveur
    soup = BeautifulSoup(response.text, 'html.parser') #On parse le résultat avec BeautifulSoup
    
    #On détermine le nombre de pages pour les reviews pro
    if len(soup.select("ul[class=pages]")) == 0 : 
        max_pages = 1
    else : 
        max_pages= int(soup.select("ul[class=pages]")[0].select("a[class=page_num]")[-1].text)

    #On instancie une liste vide dans laquelle on stockera les infos de chaque reviews    
    crit_review=[] 
    
    for page in range(0,max_pages): # On parcours les pages des reviews     
        if page > 0: url= url+'&page='+str(page) # S'il y en a plusieurs on met à jour l'url
        user_agent = {'User-agent': 'Mozilla/5.0'}
        response  = requests.get(url, headers = user_agent, timeout=60) # On requête le serveur
        soup = BeautifulSoup(response.text, 'html.parser') #On parse le résultat avec BeautifulSoup
        
        # On parcours toutes les critiques de la page 
        for review in soup.find_all('div', class_='review_content'):
            
            if review.find('div', class_='source') == None: break # Si on ne trouve rien on s'arrête
            
            #Récupération de la date de la review
            if review.select("div[class^=date]") != []: #Si on trouve une balise avec la date 
                review_date=review.select("div[class^=date]")[0].text # On l'affecte à la variable review_date
            else : review_date='None' # Sinon on affecte None à review_date
                            
            review_name=review.select("div[class^=source]")[0].text.strip('\n')  # Récupération du nom du critique

            review_score=review.select("div[class^=metascore_w]")[0].text  # Récupération du score donné par le critique
            
            # Récupération du commentaire du critique
            if review.find('span', class_='blurb blurb_expanded'):
                review_body=review.find('span', class_='blurb blurb_expanded').text
            else:
                review_body=review.select("div[class^=review_body]")[0].text.strip()
                
            crit_review.append([review_date,review_name,review_score,review_body]) # On ajoute les infos à la liste crit_review

    return crit_review # On retourne la liste crit_review avec toutes reviews de la/les pages


##################################################################
########## Fonction de récupération des critiques pro  ###########
##################################################################

def user_reviews(url):  
    # Fonction de récupération des meta reviews (critiques utilisateurs) 
   
    url= url+'/user-reviews'  # on se place sur la page des critques users
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent, timeout=60) # On requête le serveur
    soup = BeautifulSoup(response.text, 'html.parser') #On parse le résultat avec BeautifulSoup
    
    #On détermine le nombre de pages pour les reviews pro
    if len(soup.select("ul[class=pages]")) == 0 : 
        max_pages = 1
    else : 
        max_pages= int(soup.select("ul[class=pages]")[0].select("a[class=page_num]")[-1].text) 
    
    #On instancie une liste vide dans laquelle on stockera les infos de chaque reviews 
    user_review=[]

    for page in range(0,max_pages): # On parcours les pages des reviews 
        url= url+'?sort-by=score&num_items=100&page='+str(page) # On se place sur l'urlen fonction à sa page
        user_agent = {'User-agent': 'Mozilla/5.0'}
        response  = requests.get(url, headers = user_agent, timeout=60) # On requête le serveur
        soup = BeautifulSoup(response.text, 'html.parser') #On parse le résultat avec BeautifulSoup
        
        # On parcours toutes les critiques de la page
        for review in soup.find_all('div', class_='review_content'):
            #Récupération de la date de la review
            if review.find('div', class_='name') == None: break # Si on ne trouve rien on s'arrête

            if review.select("div[class^=date]") != []:#Si on trouve une balise avec la date
                review_date=review.select("div[class^=date]")[0].text # On l'affecte à la variable review_date
            else : review_date='None' # Sinon on affecte None à review_date
                            
            review_name=review.select("div[class^=name]")[0].text.strip('\n') # Récupération du nom du user
            
            review_score=review.select("div[class^=metascore_w]")[0].text  # Récupération du score donné par le user
            
            # Récupération du commentaire du critique
            if review.find('span', class_='blurb blurb_expanded'):
                review_body=review.find('span', class_='blurb blurb_expanded').text
            else:
                review_body=review.select("div[class^=review_body]")[0].text.strip()
                
            user_review.append([review_date,review_name,review_score,review_body]) # On ajoute les infos à la liste user_review

    return user_review # On retourne la liste user_review avec toutes reviews de la/les pages


##################################################################
####### Fonction de parcours des jeu sur la page active  #########
##################################################################

def scrap_page_title(url,start_page,start_title,nb_p):
    # Fonction de scraping de la page de recherche de résultat 

    for p in range(start_page,nb_p):
        print(p)
        url= url+'&page='+str(p)
        headermap = {"User-Agent": "Mac Firefox"};
        markup = requests.get(url, headers=headermap).text
        soup= BeautifulSoup(markup,"lxml")
        

        games_list= soup.select('td',{'class' : 'clamp-image-wrap'})
        games_list=[elt for idx, elt in enumerate(games_list) if idx % 2 == 0]

        for game in games_list[start_title:]:   
            game_url= 'https://www.metacritic.com'+game.find('a',href=True).attrs['href']
            #games_url.append(game_url)
            print(game_url)
            if game_url in broken_link:
                continue
            else :
                headermap = {"User-Agent": "Mac Firefox"};
                game_markup = requests.get(game_url, headers=headermap).text
                sleep(2)
                game_soup= BeautifulSoup(game_markup,"lxml")                
                
                p_name,p_platform,p_publisher,p_year,p_developer,p_genre,p_metascore,p_userscore,p_n_pro, p_n_user= retrieve_data(game_soup)
                
                gname.append(p_name)

                platform.append(p_platform)

                publisher.append(p_publisher)

                year.append(p_year)


                developer.append(p_developer)

                genre.append(p_genre)

                meta_score.append(p_metascore)

                user_score.append(p_userscore)

                p_metacrit= critic_reviews(game_url)                
                pro_critics.append(p_metacrit)
                len_pro_critics.append(len(p_metacrit))
                len_pro_critics.append(p_n_pro)

                p_usercrit= user_reviews(game_url)
                user_critics.append(p_usercrit)
                len_user_critics.append(len(p_usercrit))
                len_user_critics.append(p_n_user)
                
                print('Name:',p_name,' -','Platform:',p_platform,' -','Publisher:',p_publisher)
                print('Year:',p_year,' -','Developer:',p_developer,' -','Metascore:',p_metascore,' -','Userscore:',p_userscore)
                print('n p_crit:',len(p_metacrit),' -','n u_crit:',len(p_usercrit))
                print('\n')
                sleep(5)
                
################################################################
#### Fonction de parcours de chaque page du site par année  ####
################################################################               
                
def metacritic_scrap(start_y,end_y):
    # Fonction (main) du scraping de metacritic
    
    for annee in range(start_y,end_y):   

        print('année:', annee)
        url= url_start+str(annee)+url_end
        headermap = {"User-Agent": "Mac Firefox"};
        markup = requests.get(url, headers=headermap).text
        soup= BeautifulSoup(markup,"lxml")

        if soup.select("a[class=page_num]") != [] :
            pages= int(soup.select("a[class=page_num]")[-1].text)
        else: pages=1

        start_page= 0
        start_title= 0
        scrap_page_title(url,start_page,start_title,pages)

        columns = {
        'Name': gname,
        'Platform': platform,
        'Year': year,
        'Genre': genre,
        'Critic_Score': meta_score,
        'User_Score': user_score,
        'Publisher': publisher,
        'Developer': developer,
        'N_proreviews': len_pro_critics,
        'N_usereviews': len_user_critics
        }
    
        df = pd.DataFrame(columns)
        df = df[['Name','Platform','Year','Genre','Critic_Score','User_Score',
             'Publisher','Developer','N_proreviews','N_usereviews']]
        
        # On génere un csv pour les jeux de l'année parcourue
        file_name='metacrit_'+str(annee)+'.csv'

        df.to_csv(file_name, sep=",", encoding='utf-8', index=False)

        df_pro= pd.DataFrame({}, columns= ['Name', 'Platform','Genre','Date_crit', 'Name_crit','Score_crit','Comment_crit'])

        # On génere un csv pour les critiques pro des jeux de l'année parcourue
        k=0
        for i in range (0,len(gname)):
            for j in range(0,len(pro_critics[i])):
                df_pro.loc[k]= [gname[i],platform[i],genre[i], pro_critics[i][j][0], pro_critics[i][j][1], pro_critics[i][j][2], pro_critics[i][j][3]]
                k+=1
            k+=1

        file_name= 'metareview_'+str(annee)+'.csv'
        df_pro.to_csv(file_name, sep=",", encoding='utf-8', index=False)

        df_user= pd.DataFrame({}, columns= ['Name', 'Platform','Genre','Date_crit', 'Name_crit','Score_crit','Comment_crit'])

        # On génere un csv pour les critiques utilisateurs des jeux de l'année parcourue
        k=0
        for i in range (0,len(gname)):
            for j in range(0,len(user_critics[i])):
                df_user.loc[k]= [gname[i],platform[i],genre[i], user_critics[i][j][0], user_critics[i][j][1], user_critics[i][j][2], user_critics[i][j][3]]
                k+=1
            k+=1

        file_name= 'usereview_'+str(annee)+'.csv'
        df_user.to_csv(file_name, sep=",", encoding='utf-8', index=False)

In [ ]:
# début de l'url du site trié par metascore et par année
url_start = 'https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=' 
# fin de l'url
url_end= '&distribution=&sort=desc&view=detailed&' 

# Liste contenant les liens 'cassés'
broken_link= ['https://www.metacritic.com/game/pc/agatha-christie-the-abc-murders',
              'https://www.metacritic.com/game/3ds/3d-fantasy-zone-ii-w',
              'https://www.metacritic.com/game/pc/no-mans-land',
              'https://www.metacritic.com/game/playstation-2/pinball-hall-of-fame-the-gottlieb-collection',
              'https://www.metacritic.com/game/pc/the-experiment',
              'https://www.metacritic.com/game/ds/naruto-shippuden-shinobi-rumble',
              'https://www.metacritic.com/game/pc/pid',
              'https://www.metacritic.com/game/pc/memento-mori-2',
              'https://www.metacritic.com/game/pc/yatagarasu-attack-on-cataclysm',
              'https://www.metacritic.com/game/pc/nightcry',
              'https://www.metacritic.com/game/xbox-one/voodoo-vince-remastered',
              'https://www.metacritic.com/game/switch/the-longing',
              'https://www.metacritic.com/game/pc/el-shaddai-ascension-of-the-metatron',
             'https://www.metacritic.com/game/pc/area-51']

In [ ]:
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
current_year = date.strftime("%Y")

In [ ]:
# On parcours les années de 1995 à l'année actuelle si les fichiers csv n'existe pas on scape les données

gname = []
platform = []
year = []
genre = []
meta_score = []
user_score= []
publisher = []
developer = []
pro_critics = []
len_pro_critics= []
user_critics= []
len_user_critics= []


for i in range(1995, int(current_year)+1):
    metacrit_year= './metacrit_'+str(i)+'.csv'
    metareview_year= 'metareview_'+str(i)+'.csv'
    usereview_year= 'usereview_'+str(i)+'.csv'
    if (os.path.exists(metacrit_year) & os.path.exists(metareview_year) & os.path.exists(usereview_year)) == False :
        metacritic_scrap(i,i+1)


In [ ]:
metacritic= pd.DataFrame()
for i in range(1995,2023): # Pas de données avant 1995
    filename= 'metacrit_'+str(i)+'.csv'
    df= pd.read_csv(filename)
    metacritic= pd.concat([metacritic,df],ignore_index=True)
metacritic.head()

# On renomme les colonnes du dataframe
metacritic= metacritic.rename(columns={'Year': 'Release_date', 'Critic_Score' : 'Meta_Critic_Score','User_Score':'Meta_User_Score',
                           'N_proreviews': 'N_Critic_Reviews', 'N_usereviews': 'N_User_Reviews' })

# On supprime les doublons
metacritic= metacritic.drop_duplicates(keep='first')

# Enregistrement du dataset nettoyé dans metacritic_clean.csv
metacritic.to_csv('metacritic.csv', sep=",", encoding='utf-8', index=False)